In [ ]:
%pip install PyCryptodome
%pip install pwntools

In [ ]:
from datetime import datetime, timedelta
import requests
from pwn import xor
from Crypto.Util.Padding import pad, unpad

In [ ]:
def seperate2blocks(data, block_size):
    return [data[i:i+block_size] for i in range(0, len(data), block_size)]

def check_admin(cookie, iv):
    return requests.get(f'https://aes.cryptohack.org/flipping_cookie/check_admin/{cookie}/{iv}/').json()['flag']

The cipher looks like:
$$C = [E(IV \oplus P_1), E(C_1 \oplus P_2), E(C_2 \oplus P_3), \dots, E(C_{n-1} \oplus P_n)] $$
where $C_i$ is the $i$-th block of the ciphertext, $P_i$ is the $i$-th block of the plaintext, and $E$ is the block cipher encryption function.
By looking at the code we know that:
$$P_1 = "\text{admin=False;expi}"$$
We want that the decrypted text will have "admin=True" in it. The decrypted first block is:
$$D_1 = D(\tilde{C_1}) \oplus \tilde{IV}$$
where we can choose both $\tilde{C_1}$ and $\tilde{IV}$. Now, let's choose $\tilde{C_1}=C_1$ amd $\tilde{IV}$ as:
$$\tilde{IV} = P_1 \oplus IV \oplus "\text{admin=True;}"$$ 
and we will get:
$$D_1 = D(C_1) \oplus P_1 \oplus IV \oplus "\text{admin=True;}" = "\text{admin=True;}"$$

In [ ]:
iv_and_cookie = bytes.fromhex(requests.get(f'https://aes.cryptohack.org/flipping_cookie/get_cookie/').json()['cookie'])
iv = iv_and_cookie[:16]
cookie = iv_and_cookie[16:]

In [ ]:
c1 = seperate2blocks(cookie, 16)[0]
new_iv = xor(iv, b"admin=False;expi", pad(b"admin=True;ex",16))
check_admin(c1.hex(), new_iv.hex())